In [1]:
!pip install wandb

In [2]:
%matplotlib inline
import random
import torch
from torch.utils import data
from d2l import torch as d2l

In [3]:
import wandb
wandb.init(project='course') # specify the project of the current run

In [4]:
true_w = torch.tensor([2, -3.4, 5, 6])
true_b = 2.4
features, labels = d2l.synthetic_data(true_w, true_b, 2000)

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 32
data_iter = load_array((features, labels), batch_size)

In [6]:
next(iter(data_iter))

[tensor([[-7.4017e-01, -3.2112e-01,  3.9978e-01,  1.8021e+00],
         [ 6.2178e-01,  2.1092e+00, -1.5599e-01, -1.1109e-01],
         [ 8.6601e-01,  7.7299e-01, -6.9430e-01, -1.0669e+00],
         [ 1.2534e+00, -3.8269e-01, -8.7639e-01, -1.5491e+00],
         [-8.7189e-01,  5.9789e-02, -2.2750e+00, -5.6527e-01],
         [-4.2654e-01,  2.1044e-01,  7.4166e-01, -5.2512e-01],
         [-4.7578e-01,  1.7765e+00,  2.9422e+00, -3.1139e-01],
         [-1.1495e-01, -4.7511e-01,  2.9386e-02,  9.1964e-01],
         [-3.2373e-02,  1.2088e-02, -1.0838e+00, -3.6136e-01],
         [-1.5382e+00,  1.4769e-01, -4.0155e-01, -2.5135e+00],
         [-2.2676e+00,  6.8901e-01,  6.0846e-02,  2.0099e+00],
         [-9.8130e-01, -5.4044e-01,  1.5668e+00,  6.2797e-01],
         [ 2.7403e+00,  7.2366e-01, -4.6050e-01, -4.6123e-01],
         [ 6.8973e-01, -2.9694e-02, -2.5740e-02,  1.4053e+00],
         [ 1.6242e-02, -1.1006e+00, -2.0717e-01, -8.4463e-01],
         [ 8.0265e-01, -2.0285e-01,  1.9061e+00, -8.868

In [7]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(4, 1))

In [8]:
net[0].weight.data.normal_(0, 0.01) # net[0] is the first layer
net[0].bias.data.fill_(0)
net[0].weight.data

tensor([[-0.0146,  0.0074,  0.0049, -0.0058]])

In [9]:
def _weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.zero_()
        
net.apply(_weights_init)
net[0].weight.data

tensor([[ 0.3785, -0.4744, -0.6180,  0.5488]])

In [10]:
loss = nn.MSELoss()

In [11]:
wandb.watch(net, log="all", criterion=loss, log_freq=1,  log_graph=(True))

In [12]:
optim = torch.optim.SGD(net.parameters(), lr=3e-2)

In [13]:
next(net.parameters())

Parameter containing:
tensor([[ 0.3785, -0.4744, -0.6180,  0.5488]], requires_grad=True)

In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        optim.zero_grad() # please don't forget!
        l.backward() # remember: You need to tell wrt to what the gradient is computed
        optim.step() # do a step in the gradient direction
    with torch.no_grad():
        l = loss(net(features), labels) 
        print(f'epoch {epoch + 1}, loss {l.item()}')
        wandb.log({'loss': l.item()}, step=epoch)

In [15]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

In [16]:
torch.save(net.state_dict(), 'my_model.pt')

In [17]:
net.state_dict()

OrderedDict([('0.weight', tensor([[ 2.0003, -3.4006,  5.0000,  5.9998]])),
             ('0.bias', tensor([2.3996]))])

In [18]:
net[0].bias.data

tensor([2.3996])

In [19]:
!pip install wandb

In [20]:
%matplotlib inline
import random
import torch
from torch.utils import data
from d2l import torch as d2l

In [21]:
import wandb
wandb.init(project='course') # specify the project of the current run